In [4]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
nltk.download('wordnet')
nltk.download('punkt')
stemmer = SnowballStemmer('russian')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans, DBSCAN

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import fowlkes_mallows_score

import seaborn as sns

from yellowbrick.text import FreqDistVisualizer, TSNEVisualizer

from sklearn.pipeline import Pipeline 
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df = pd.read_csv('labeled.csv', encoding="utf-8")

In [6]:
df['toxic'] = df['toxic'].apply(lambda x: 'Toxic' if x==1.0 else 'Not Toxic')

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def clean_df(df):
    df = df.str.lower()
    df = df.replace(r'[^а-яА-Я]', ' ', regex=True)
    df = df.str.strip()
    df = df.apply(lambda x:' '.join([word for word in x.split() if word not in stopwords.words('russian') and len(word)>2]))
    return df

df['comment'] = clean_df(df['comment'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
!pip install pymorphy2
import pymorphy2
df = df.dropna(subset=['comment'])
morph = pymorphy2.MorphAnalyzer()

def lem_tok(text):
    text_lem = [morph.parse(word)[0].normal_form for word in text.split(' ')]
    return text_lem

In [9]:
%%time
#from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf = TfidfVectorizer(sublinear_tf=True,
                        #ngram_range=(1, 2), 
                        #tokenizer=lem_tok)

# We transform each complaint into a vector
#features = tfidf.fit_transform(df['comment'])#.toarray()

#labels = df.category_id

#print("Each of the %d complaints is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2), tokenizer=lem_tok)
features = tfidf.fit_transform(df.comment).toarray()


CPU times: user 57.2 s, sys: 1.73 s, total: 58.9 s
Wall time: 1min 25s


In [10]:
# labels = df.category_id

# df['category_id'] = df['toxic'].factorize()[0]
# category_id_df = df[['toxic', 'category_id']].drop_duplicates()
# # Dictionaries for future use
# category_to_id = dict(category_id_df.values)
# id_to_category = dict(category_id_df[['category_id', 'toxic']].values)

# # New dataframe
# df = df.sample(frac=0.2)

In [11]:
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.pipeline import Pipeline

logreg = Pipeline([
                ('tfidf', TfidfVectorizer(sublinear_tf=True,
                        ngram_range=(1, 3),
                        tokenizer=lem_tok, max_df=1000)),
                ('clf', LinearSVC()),
])

logreg.fit(df['comment'], df['toxic'])


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=1000, ngram_range=(1, 3),
                                 sublinear_tf=True,
                                 tokenizer=<function lem_tok at 0x7f59a99747a0>)),
                ('clf', LinearSVC())])

'Not Toxic'